In [17]:
%pip install -q langchainhub langchain_chroma sentence_transformers datasets langchain langchain-openai chromadb

In [18]:
pip install beautifulsoup4 langchain_community

In [19]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()
docs

[Document(page_content="\n\n\n\n\nLangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith\n\n\n\n\n\n\n\nSkip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookThis is outdated documentation for \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith, which is no longer actively maintained.For up-to-date documentation, see the latest version.User GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and oth

# Embedding

In [20]:
from datasets import load_dataset
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

from tqdm.auto import tqdm

ef = SentenceTransformerEmbeddingFunction(device="cuda")

chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="marmikpandya", embedding_function=ef)


UniqueConstraintError: Collection marmikpandya already exists

In [99]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

# documents[0].metadata['description']

inputs =[]
outputs = []

for doc in documents:
  inputs.append(doc.page_content)


In [100]:
%%time
collection.add(
    documents=inputs,
    ids=[str(i) for i in range(len(inputs))]
)

CPU times: user 46.9 ms, sys: 939 µs, total: 47.9 ms
Wall time: 46.6 ms


In [101]:
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [102]:
ef_lc = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [103]:
langchain_chroma = Chroma(
    client=chroma_client,
    collection_name="marmikpandya",
    embedding_function=ef_lc,
)
retriever = langchain_chroma.as_retriever()
langchain_chroma._collection.count()

13358

In [104]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", openai_api_key="sk-proj-5iFQiwQPizVdIZhtMp2HT3BlbkFJZLCmNvFVOC0rTy30vb2f")

In [114]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}"""

)

document_chain = create_stuff_documents_chain(llm, prompt)

In [107]:


from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])


In [109]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can help visualize test results.'

In [110]:
Q="What happened to me?"


In [111]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [112]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

Based on the provided context, it does not seem that langsmith can help with testing. The focus of the text is on difficulties with communication and setting boundaries in a relationship.
